In [2]:
import pandas as pd
import numpy as np

# -----------------------
# LOAD
# -----------------------
orders = pd.read_csv("orders.csv")
order_lines = pd.read_csv("order_lines.csv")
shipments = pd.read_csv("shipments.csv")
shipment_lines = pd.read_csv("shipment_lines.csv")

# -----------------------
# PARSE DATES
# -----------------------
orders["order_date"] = pd.to_datetime(orders["order_date"])
orders["promised_delivery_date"] = pd.to_datetime(orders["promised_delivery_date"])

shipments["ship_date"] = pd.to_datetime(shipments["ship_date"])
shipment_lines["delivered_date"] = pd.to_datetime(shipment_lines["delivered_date"])

# -----------------------
# ENRICH shipment_lines with order_id + warehouse + carrier + ship_date
# -----------------------
sl = shipment_lines.merge(
    shipments[["shipment_id", "order_id", "warehouse", "carrier", "ship_date", "expedite_flag"]],
    on="shipment_id",
    how="left"
)

# -----------------------
# Aggregate delivery by order_id + sku (handles split shipments)
# -----------------------
deliv_order_sku = (
    sl.groupby(["order_id", "sku"], as_index=False)
      .agg(
          shipped_qty=("shipped_qty", "sum"),
          delivered_qty=("delivered_qty", "sum"),
          delivered_date=("delivered_date", "max"),   # last arrival for that sku in order
          damaged_flag=("damaged_flag", "max"),
          returned_flag=("returned_flag", "max")
      )
)

# Order-level delivered date = max delivered across all lines/shipments
order_delivered = (
    sl.groupby("order_id", as_index=False)
      .agg(order_delivered_date=("delivered_date", "max"))
)

# Choose "owner" (warehouse/carrier) for the order as the latest shipment (common ops convention)
order_owner = (
    shipments.sort_values(["order_id", "ship_date"])
             .groupby("order_id", as_index=False)
             .tail(1)[["order_id", "warehouse", "carrier"]]
)

# -----------------------
# Join demand + delivery at line level
# -----------------------
line = order_lines.merge(deliv_order_sku, on=["order_id", "sku"], how="left")

# Clean dataset guarantees no nulls here, but safe fill anyway
for c in ["shipped_qty", "delivered_qty"]:
    line[c] = line[c].fillna(0)

for c in ["damaged_flag", "returned_flag"]:
    line[c] = line[c].fillna(0).astype(int)

# Line KPIs
line["in_full_line_flag"] = (line["delivered_qty"] >= line["ordered_qty"]).astype(int)
line["short_qty"] = np.maximum(line["ordered_qty"] - line["delivered_qty"], 0)

line["line_revenue"] = line["ordered_qty"] * line["unit_price"]
line["delivered_revenue"] = np.minimum(line["delivered_qty"], line["ordered_qty"]) * line["unit_price"]
line["lost_revenue_est"] = line["short_qty"] * line["unit_price"]

# -----------------------
# Roll up to order level
# -----------------------
order_if = (
    line.groupby("order_id", as_index=False)
        .agg(
            in_full_order_flag=("in_full_line_flag", "min"),  # all lines must be in-full
            any_damage_flag=("damaged_flag", "max"),
            any_return_flag=("returned_flag", "max"),
            order_revenue=("line_revenue", "sum"),
            delivered_revenue=("delivered_revenue", "sum"),
            lost_revenue_est=("lost_revenue_est", "sum"),
            lines_count=("sku", "nunique"),
            total_ordered_qty=("ordered_qty", "sum"),
            total_delivered_qty=("delivered_qty", "sum"),
        )
)

order_kpi = (
    orders.merge(order_delivered, on="order_id", how="left")
          .merge(order_if, on="order_id", how="left")
          .merge(order_owner, on="order_id", how="left")
)

# On-Time + OTIF + Perfect Order
order_kpi["on_time_flag"] = (order_kpi["order_delivered_date"] <= order_kpi["promised_delivery_date"]).astype(int)
order_kpi["otif_flag"] = ((order_kpi["on_time_flag"] == 1) & (order_kpi["in_full_order_flag"] == 1)).astype(int)
order_kpi["perfect_order_flag"] = (
    (order_kpi["otif_flag"] == 1) &
    (order_kpi["any_damage_flag"] == 0) &
    (order_kpi["any_return_flag"] == 0)
).astype(int)

# Cycle time metrics
# Earliest ship date per order (order -> ship)
order_first_ship = (
    shipments.groupby("order_id", as_index=False)
             .agg(first_ship_date=("ship_date", "min"))
)

order_kpi = order_kpi.merge(order_first_ship, on="order_id", how="left")

order_kpi["warehouse_processing_days"] = (
    order_kpi["first_ship_date"] - order_kpi["order_date"]
).dt.days

# optional: drop helper column after
order_kpi.drop(columns=["first_ship_date"], inplace=True)


order_kpi["order_cycle_days"] = (order_kpi["order_delivered_date"] - order_kpi["order_date"]).dt.days

# -----------------------
# Lane KPIs (Warehouse x Carrier x Region)
# -----------------------
lane_kpi = (
    order_kpi.groupby(["warehouse", "carrier", "customer_region"], as_index=False)
            .agg(
                orders=("order_id", "nunique"),
                otif=("otif_flag", "mean"),
                on_time=("on_time_flag", "mean"),
                in_full=("in_full_order_flag", "mean"),
                perfect_order=("perfect_order_flag", "mean"),
                avg_cycle_days=("order_cycle_days", "mean"),
                lost_revenue=("lost_revenue_est", "sum"),
                revenue=("order_revenue", "sum"),
            )
)

# -----------------------
# Daily warehouse KPIs (ops throughput + service)
# -----------------------
order_kpi["order_day"] = order_kpi["order_date"].dt.date

warehouse_daily = (
    order_kpi.groupby(["warehouse", "order_day"], as_index=False)
            .agg(
                orders=("order_id", "nunique"),
                otif=("otif_flag", "mean"),
                on_time=("on_time_flag", "mean"),
                in_full=("in_full_order_flag", "mean"),
                perfect_order=("perfect_order_flag", "mean"),
                avg_cycle_days=("order_cycle_days", "mean"),
                lost_revenue=("lost_revenue_est", "sum"),
            )
)

# Carrier daily KPIs
carrier_daily = (
    order_kpi.groupby(["carrier", "order_day"], as_index=False)
            .agg(
                orders=("order_id", "nunique"),
                otif=("otif_flag", "mean"),
                on_time=("on_time_flag", "mean"),
                in_full=("in_full_order_flag", "mean"),
                perfect_order=("perfect_order_flag", "mean"),
                avg_cycle_days=("order_cycle_days", "mean"),
                lost_revenue=("lost_revenue_est", "sum"),
            )
)

# -----------------------
# EXPORT TABLEAU-READY FILES
# -----------------------
order_kpi.to_csv("kpi_order_level.csv", index=False)
line.to_csv("kpi_line_level.csv", index=False)
lane_kpi.to_csv("kpi_lane_level.csv", index=False)
warehouse_daily.to_csv("kpi_warehouse_daily.csv", index=False)
carrier_daily.to_csv("kpi_carrier_daily.csv", index=False)

print("✅ Exported KPI datasets for Tableau:")
print(" - kpi_order_level.csv")
print(" - kpi_line_level.csv")
print(" - kpi_lane_level.csv")
print(" - kpi_warehouse_daily.csv")
print(" - kpi_carrier_daily.csv")

# Quick sanity
otif = order_kpi["otif_flag"].mean()
perfect = order_kpi["perfect_order_flag"].mean()
print(f"\nSanity check: OTIF={otif:.3f} ({otif*100:.1f}%), Perfect={perfect:.3f} ({perfect*100:.1f}%)")


✅ Exported KPI datasets for Tableau:
 - kpi_order_level.csv
 - kpi_line_level.csv
 - kpi_lane_level.csv
 - kpi_warehouse_daily.csv
 - kpi_carrier_daily.csv

Sanity check: OTIF=0.613 (61.3%), Perfect=0.457 (45.7%)
